In [2]:
import QuantLib as ql

# Analytic continuous geometric average-price Asians

In [3]:
dc = ql.Actual360()

In [4]:
today = ql.Settings.instance().evaluationDate

In [9]:
spot = ql.SimpleQuote(80.0)
qRate = ql.SimpleQuote(-0.03)
qTS = ql.FlatForward(today, ql.QuoteHandle(qRate), dc)
rRate = ql.SimpleQuote(0.05)
rTS = ql.FlatForward(today, ql.QuoteHandle(rRate), dc)
vol = ql.SimpleQuote(0.20)
volTS = ql.BlackConstantVol(today, ql.NullCalendar(), ql.QuoteHandle(vol), dc)

In [11]:
stochProcess = ql.BlackScholesMertonProcess(ql.QuoteHandle(spot), ql.YieldTermStructureHandle(qTS), 
                                            ql.YieldTermStructureHandle(rTS), 
                                            ql.BlackVolTermStructureHandle(volTS))

In [12]:
engine = ql.AnalyticContinuousGeometricAveragePriceAsianEngine(stochProcess)

In [13]:
averageType = ql.Average.Geometric

In [15]:
option_type = ql.Option.Put

In [16]:
strike = 85.0
exerciseDate = today + 90

In [17]:
pastFixings = ql.nullInt()

In [18]:
runningAccumulator = ql.nullDouble()

In [20]:
payoff = ql.PlainVanillaPayoff(option_type, strike)

In [21]:
exercise = ql.EuropeanExercise(exerciseDate)

In [22]:
option = ql.ContinuousAveragingAsianOption(averageType, payoff, exercise)

In [23]:
option.setPricingEngine(engine)

In [24]:
calculated = option.NPV()

In [25]:
expected = 4.6922

In [26]:
tolerance = 1.0e-4

In [27]:
if (abs(calculated-expected) > tolerance):
    print("FAILURE")

# Analytic continuous geometric average-price Asian greeks

In [30]:
calculated, expected, tolerance = {}, {}, {}

In [31]:
tolerance["delta"]  = 1.0e-5
tolerance["gamma"]  = 1.0e-5
tolerance["theta"]  = 1.0e-5
tolerance["rho"]    = 1.0e-5
tolerance["divRho"] = 1.0e-5
tolerance["vega"]   = 1.0e-5

In [32]:
types = [ql.Option.Call, ql.Option.Put]

In [33]:
underlyings = [100.0]

In [34]:
strikes = [90.0, 100.0, 110.0]

In [35]:
qRates = [0.04, 0.05, 0.06]

In [37]:
rRates = [0.01, 0.05, 0.15]

In [38]:
lengths = [1, 2]

In [39]:
vols = [0.11, 0.50, 1.20]

In [40]:
dc = ql.Actual360()

In [41]:
today = ql.Settings.instance().evaluationDate

In [42]:
ql.Settings.instance().evaluationDate = today

In [82]:
spot = ql.SimpleQuote(0.0)
qRate = ql.SimpleQuote(0.0)
qTS = ql.YieldTermStructureHandle(ql.FlatForward(0, ql.NullCalendar(), ql.QuoteHandle(qRate), dc))
rRate = ql.SimpleQuote(0.0)
rTS = ql.YieldTermStructureHandle(ql.FlatForward(0, ql.NullCalendar(), ql.QuoteHandle(rRate), dc))
vol = ql.SimpleQuote(0.0)
volTS = ql.BlackVolTermStructureHandle(ql.BlackConstantVol(0, ql.NullCalendar(), ql.QuoteHandle(vol), dc))

In [83]:
process = ql.BlackScholesMertonProcess(ql.QuoteHandle(spot), qTS, 
                                            rTS, 
                                            volTS)

In [84]:
def relativeError(x1, x2, reference):
        if (reference != 0.0):
            return abs(x1-x2)/reference
        else:
            # fall back to absolute error
            return abs(x1-x2)

In [85]:
def REPORT_FAILURE(greekName, averageType, runningAccumulator, pastFixings,
                       fixingDates, payoff, exercise, s, q, r, today, v,
                       expected, calculated, tolerance):
    print (str(exercise)
                                        + " Asian option with "
                                        + str(averageType) + " and "
                                        + str(payoff) + " payoff:\n"

                                        + "    future fixings:   " + str(len(fixingDates)) + "\n"
                                        + "    underlying value: " + str(s) + "\n"
                                        + "    strike:           " + str(payoff.strike()) + "\n"
                                        + "    dividend yield:   " + str(q) + "\n"
                                        + "    risk-free rate:   " + str(r) + "\n"
                                        + "    reference date:   " + str(today) + "\n"
                                        + "    maturity:         " + str(exercise.lastDate()) + "\n"
                                        + "    volatility:       " + str(v) + "\n\n"
                                        + "    expected   " + greekName + ": " + str(expected) + "\n"
                                        + "    calculated " + greekName + ": " + str(calculated) + "\n"
                                        + "    error:            " + str(abs(expected-calculated))
                                        + "\n"
                                        + "    tolerance:        " + str(tolerance))

In [93]:
2*ql.Years

TypeError: 'int' object is not callable

In [90]:
today + length*ql.Years


Date(19,4,2019)

In [94]:
for option_type in types:
    for strike in strikes:
        for length in lengths:
            maturity = ql.EuropeanExercise(today + ql.Period(length,ql.Years))
            payoff = ql.PlainVanillaPayoff(option_type, strike)
            engine = ql.AnalyticContinuousGeometricAveragePriceAsianEngine(process)
            option = ql.ContinuousAveragingAsianOption(ql.Average.Geometric, payoff, maturity)
            option.setPricingEngine(engine)
            pastFixings = ql.nullInt()
            runningAccumulator = ql.nullDouble()
            for u in underlyings:
                for q in qRates:
                    for r in rRates:
                        for v in vols:
                            spot.setValue(u)
                            qRate.setValue(q)
                            rRate.setValue(r)
                            vol.setValue(v)
                            value = option.NPV()
                            calculated["delta"]  = option.delta()
                            calculated["gamma"]  = option.gamma()
                            calculated["theta"]  = option.theta()
                            calculated["rho"]    = option.rho()
                            calculated["divRho"] = option.dividendRho()
                            calculated["vega"]   = option.vega()
                            if (value > spot.value()*1.0e-5):
                                du = u*1.0e-4
                                spot.setValue(u+du)
                                value_p = option.NPV()
                                delta_p = option.delta()
                                spot.setValue(u-du)
                                value_m = option.NPV()
                                delta_m = option.delta()
                                spot.setValue(u)
                                expected["delta"] = (value_p - value_m)/(2*du)
                                expected["gamma"] = (delta_p - delta_m)/(2*du)
                                
                                dr = r*1.0e-4
                                rRate.setValue(r+dr)
                                value_p = option.NPV()
                                rRate.setValue(r-dr)
                                value_m = option.NPV()
                                rRate.setValue(r)
                                expected["rho"] = (value_p - value_m)/(2*dr)
                                
                                dq = q*1.0e-4
                                qRate.setValue(q+dq)
                                value_p = option.NPV()
                                qRate.setValue(q-dq)
                                value_m = option.NPV()
                                qRate.setValue(q)
                                expected["divRho"] = (value_p - value_m)/(2*dq)
                                
                                dv = v*1.0e-4
                                vol.setValue(v+dv)
                                value_p = option.NPV()
                                vol.setValue(v-dv)
                                value_m = option.NPV()
                                vol.setValue(v)
                                expected["vega"] = (value_p - value_m)/(2*dv)
                                
                                dT = dc.yearFraction(today-1, today+1)
                                ql.Settings.instance().evaluationDate = today - 1
                                value_m = option.NPV()
                                ql.Settings.instance().evaluationDate = today + 1
                                value_p = option.NPV()
                                ql.Settings.instance().evaluationDate = today
                                expected["theta"] = (value_p - value_m)/dT
                                
                                for greek in calculated.keys():
                                    expct = expected  [greek]
                                    calcl = calculated[greek]
                                    tol   = tolerance [greek]
                                    error = relativeError(expct,calcl,u)
                                    
                                    if (error>tol):
                                        REPORT_FAILURE(greek, ql.Average.Geometric,
                                                 None, None,
                                                 [],
                                                 payoff, maturity,
                                                 u, q, r, today, v,
                                                 expct, calcl, tol)

                                